## Importing Libraries

In [2]:
import pandas as pd
import scipy.io as sio
import numpy as np
import math
import pprint
from scipy.stats import norm

## Data Pre-processing

The purpose of this cell is to select all of the data from the master excel sheet that is difficult to handle and output this data to a new excel sheet to be further altered and added to. Once ran once, this cell is pretty much rendered useless because it would overwrite any additions you make on the target output excel sheet.

In [1]:
# Importing master excel document and extracting relevant data
data_raw = pd.read_excel('Input_Final_LIBSRATE.xlsx', 'Input', header = 4)
auto_data = data_raw.drop(list(range(0, 9)))
auto_data.drop(list(auto_data.columns)[82:], axis = 1)
desired_columns = ['Parameters:', 'Population (low)', 'CpC (baseline)', 'S1.2', 'S1.3', 'S1.4', 'S1.5', 'S1.6', 'S1.7', 'S1.8', 'S1.9', 'S1.10', 'S1.11', 'S1.12', 'S1.13', 'S1.14']
auto_data = pd.concat([auto_data[col] for col in auto_data.columns if col in desired_columns], axis = 1)
auto_data.columns = ['Year', 'Population', 'CpC', 'Closures', 'Bumpers', 'Engine Blocks', 'Heat Exchangers', 'Cylinder Heads', 'Suspension', 'Steering', 'Wheels', 'Transmission/Driveline', 'Brake Componenets', 'Other Engine', 'Body', 'Other Components']
auto_data.set_index('Year')
writer = pd.ExcelWriter('LIBS Rate ipynb Output Data.xlsx')
auto_data.to_excel(writer, 'Auto Data', index = False, index_label = 'Year')

# Importing alloy information
alloys_raw = sio.loadmat('alloys_info.mat')

# Parsing .MAT raw data into hierarchical array representation
# [alloy, type, [nominal[], min[], max[]]]
alloys_info = []
for alloy in alloys_raw['alloys_info'][0]:
    alloys_info.append([alloy[0][0][0][0], alloy[3][0], 'Nominal', *list(alloy[1][0])])
    alloys_info.append([alloy[0][0][0][0], alloy[3][0], 'Minimum', *list(alloy[2][1])])
    alloys_info.append([alloy[0][0][0][0], alloy[3][0], 'Maximum', *list(alloy[2][0])])
alloys_info = pd.DataFrame(alloys_info)
alloys_info.columns = ['Alloy', 'Type', 'Level', 'Si', 'Fe', 'Cu', 'Mn', 'Mg', 'Cr', 'Ni', 'Zn', 'Ti']
alloys_info_table = pd.DataFrame(alloys_info)
#alloys_info_table = pd.pivot_table(alloys_info, 
#                       index = ['Alloy', 'Type', 'Level'],
#                       values = ['Level', 'Si', 'Fe', 'Cu', 'Mn', 'Mg', 'Cr', 'Ni', 'Zn', 'Ti'])
alloys_info_table.to_excel(writer, 'Alloy Data', index = False)
alloys_info.set_index('Alloy', drop = True)#['Alloy', 'Type', 'Level'], drop = True)

writer.save()

NameError: name 'pd' is not defined

## Set Scenario Conditions, Parameters, and Wrangle Data

TODO:
- Create function to input parameters for scenarios
- Add parameters for starting year, and whatever criterion will be dynamic between this model and the regional ones

In [3]:
##### SCENARIO CONDITIONS #####
# Population: 1 = baseline;
population_scen = 1 # PQ

# Cars per capita: 1 = baseline
cars_per_capita_scen = 1 # CpCQ

# Dismantling before ELVyear: 0 = none, 1 = low, 2 = high, 3 = all
ELV_year = 1990 # ELVyear
dismant_pre_ELV_scen = 0 # ELVQ1

# Dismantling from ELVyear: 0 = none, 1 = low, 2 = high, 3 = all
dismant_post_ELV_scen = 1 # ELVQ2

# Alloy sorting scenario: 0 = none, 2 = laser sorting
sorting_scen = 2 # ZQ

# LIBS implmentation rate: 0 = none, 1 = low, 2 = medium, 3 = immediate, 100%
LIBS_rate_scen = 1 #LIBSrate

# Lifetime scenario: 2 = baseline, 1 = low, 3 = high
lifetime_scen = 2 #LTQ

# Demagging: 1 = on, 2 = off
demag_scen = 1 #DMGQ

# Zorba scenario: 0 = current, 1 = 2% reduction
zorba_export_scen = 0 # Zorba_export

##### PARAMETERS #####
# Number of alloy groups that are sorted, e.g. 1xxx, 3xxx
num_sorted_alloy_groups = 8 # NAG

# This parameter reduces the amount of scrap that can be intelligently sorted as empirical data 
# has been collected that indicates that approximately 25% of scrap partiuculates are under 1 inch
sizelimited = 0.25

##### Variables #####

# importing all sheets from excel sheet: 'LIBS Rate Input Data.xlsx'
data = pd.read_excel('LIBS Rate Input Data.xlsx', sheet_name = None)
old_scrap = data['Old Scrap']                                                                               # other_old_scrap
old_scrap_comp = data['Old Alloy Compositions']                                                             # other_comp
laser_sorting_alloy_groups = data['Laser Sorting Alloy Groups']                                             # alloy_names
#####################################################################

alloy_data = data['Alloy Data'].reset_index(drop = True)
#alloy_data = pd.MultiIndex.from_arrays(data['Alloy Data'].values)

#####################################################################
auto_data = data['Auto Data']
scenario_data = data['Scenarios']
group_comp_by_alloy = data['Group Composition by Alloy']                                                    # AR_Ga
group_comp_by_alloy.index = group_comp_by_alloy['Component']
group_comp_by_alloy = group_comp_by_alloy.drop(columns = ['Component'])
hand_sort = data['Hand Sorting']                                                                            # ZR_RaZ(:,:,1)       
laser_sort = data['Laser Sorting']                                                                          # ZR_RaZ(:,:,2)
primary_metal_comp = data['Primary Metal Composition']                                                      # CP_Ee
element_yields = data['Element Yields']
component_yields = data['Componenent Yields']
opt_costs = data['Optimization Costs']
manufacture_scrap_recipe = data['Manufacturing Scrap Recipe'].transpose()                                   # MSR_Gr

# creating variables for names of headers
alloy_names = list(old_scrap_comp.index)                                                                    # AlloyNames
element_names = list(data['Old Alloy Compositions'].columns)                                                # ElementNames
raw_material_names = list(data['Laser Sorting Alloy Groups'].index)                                         # RawMaterialNames
comp_group_names = list(data['Auto Data'].columns)[3:]                                                      # GroupNames
years = list(data['Auto Data']['Year'])                                                                     # Time

# creating variables for number of columns for each feature
num_raw_materials = len(raw_material_names)                                                                 # NR
num_alloys = len(alloy_names)                                                                               # NA
num_elements = len(element_names)                                                                           # NE
num_component_groups = len(comp_group_names)                                                                # NG
num_years = len(years)                                                                                      # NT
num_segments = 1                                                                                            # NS
num_cohorts = num_years # subject to change                                                                 # NC

# creating misc variables
# TODO: change values to be from the 'Scenarios' sheet of 'LIBS Rate Input Data'
exp_life = 16                                                                                               # Mu
std_dev = 3                                                                                                 # Sigma
zorba_export_rate = [.45] * num_years
collection_rate = [.98] * num_years                                                                         # CR_T
shredder_yield = .95                                                                                        # SY
secondary_alloys = True                                                                                     # SecondaryAlloysIndex
primary_alloys = not secondary_alloys                                                                       # PrimaryAlloysIndex

# importing data from within excel sheets
population = auto_data['Population']                                                                        # P_T
cars_per_capita = auto_data['CpC'] / 1000                                                                   # CpC_T
segmentation = [1] * num_years # always 100% on spreadsheet (fraction of input)                             # SGM_Ts
avg_Al_weight_per_comp = auto_data[list(auto_data.columns)[3:]]                                             # GW_TSG
LIBS_rate = scenario_data['LIBS Rate' + str(LIBS_rate_scen)]                                                # LIBSrate
ELV_rate_pre = data['ELV Scenario'].iloc[:, dismant_pre_ELV_scen]
ELV_rate_post = data['ELV Scenario'].iloc[:, dismant_post_ELV_scen]
ELV_year_index = scenario_data.Year[scenario_data.Year == ELV_year].index[0]
ELV_by_comp = pd.DataFrame(np.concatenate((np.ones((ELV_year_index,1)).dot(ELV_rate_pre.to_frame().T),      # ELV_TG
                                           np.ones((num_years - ELV_year_index,1)).
                                           dot(ELV_rate_post.to_frame().T)), axis = 0),
                           columns = list(auto_data.columns)[3:])                                                  
sorting_rates = scenario_data[['Sorting' + str(sorting_scen) + ' Hand',                                     # Z_Tz
                               'Sorting' + str(sorting_scen) + ' Laser']]     
remelting_yields = element_yields.loc['Remelting Yields']                                                   # RY
shredder_cont = element_yields.loc['Shredder Contamination']                                                # SC_e
shredder_cont_dism = element_yields.loc['Shredder Contamination with Dismantled Parts']                     # SCD_e
manufacture_yields = component_yields.loc['Manufacturing']                                                  # MY_G
alloy_opt_costs = opt_costs.loc['Optimization Cost']                                                        # H_R

## Variable Initialization

In [125]:
# X01 - Kg
X01_TR = np.zeros((num_years, num_raw_materials))

# X07a - Kg
X07a_TE = np.zeros((num_years, num_elements))

# X07b - Kg
X07b_TE = np.zeros((num_years, num_elements))

# X10 - Kg
X10_TR = np.zeros((num_years, num_raw_materials))

# X12 - Kg
X12_TAE = np.zeros((num_years, num_alloys, num_elements))
X12_TRA = np.zeros((num_years, num_raw_materials, num_alloys))
X12_TR = np.zeros((num_years, num_raw_materials))
X12_RAE = np.zeros((num_raw_materials, num_alloys, num_elements))

# X20 - Kg
X20_TAE = np.zeros((num_years, num_alloys, num_elements))

# X23 - Kg
X23_TAE = np.zeros((num_years, num_alloys, num_elements))
X23_conc_TAE = np.zeros((num_years, num_alloys, num_elements))
X23_TA = np.zeros((num_years, num_alloys))
X23_TG = np.zeros((num_years, num_component_groups))
X23_TGA = np.zeros((num_years, num_component_groups, num_alloys))
X23_TGE = np.zeros((num_years, num_component_groups, num_elements))                                             # X23_TGe

# X31 - Kg
X301_TG = np.zeros((num_years, num_component_groups))
X31_TRE = np.zeros((num_years, num_raw_materials, num_elements))
X31_TAR = np.zeros((num_years, num_alloys, num_raw_materials))
X301_TGA = np.zeros((num_years, num_component_groups, num_alloys))

# X34 - Kg
X34_TG = np.zeros((num_years, num_component_groups))

# X40 - Export
X40_TG = np.zeros((num_years, num_component_groups))
X40_T = np.zeros((num_years, 1))

# X45 - Cars, Kg
X45_TS = np.zeros((num_years, num_segments)) # Cars
X45_TG = np.zeros((num_years, num_component_groups))
X45_T = np.zeros((num_years, 1))

# X50 - Kg
X50_TCG = np.zeros((num_years, num_cohorts, num_component_groups))

# X56 - Kg
X56_TCG = np.zeros((num_years, num_cohorts, num_component_groups))

# X560 - Cars, Kg
X560_T = np.zeros((num_years, 1))
X560_TC = np.zeros((num_years, num_cohorts)) # Cars
X560_TCG = np.zeros((num_years, num_cohorts, num_component_groups))
X560_TCA = np.zeros((num_years, num_cohorts, num_alloys))
X560_TCE = np.zeros((num_years, num_cohorts, num_elements))
X560_TE = np.zeros((num_years, num_elements))

# X67a - Kg
X67a_TCG = np.zeros((num_years, num_cohorts, num_component_groups))
X67a_TCGA = np.zeros((num_years, num_cohorts, num_component_groups, num_alloys))
X67a_TCGE = np.zeros((num_years, num_cohorts, num_component_groups, num_elements))
X67a_TGE = np.zeros((num_years, num_component_groups, num_elements))
X67b_TA = np.zeros((num_years, num_alloys))

# X67b - Kg
X67b_TCA = np.zeros((num_years, num_cohorts, num_alloys))
X67b_TAE = np.zeros((num_years, num_alloys, num_elements))
X67b_TCAE = np.zeros((num_years, num_component_groups, num_alloys, num_elements))
X67b_TCGE = np.zeros((num_years, num_cohorts, num_component_groups, num_elements))
X67b_TAE = np.zeros((num_years, num_alloys, num_elements))
X67b_T = np.zeros((num_years, 1))                                                                               # X67a_otherT
X67b_TA = np.zeros((num_years, num_alloys))                                                                     # X67a_otherTA
X67b_TAE_other = np.zeros((num_years, num_alloys, num_elements))                                                # X67b_otherTAE

# X7a1 - Kg
X7a1_TRE = np.zeros((num_years, num_raw_materials, num_elements))
X7a1_TGE = np.zeros((num_years, num_component_groups, num_elements))

# X7b1 - Kg
X7b1_TRE = np.zeros((num_years, num_raw_materials, num_elements))
X7b1b_TRE = np.zeros((num_years, num_raw_materials, num_elements))
X7b1_TRE_nocont = np.zeros((num_years, num_raw_materials, num_elements))
X7b1s_TRE = np.zeros((num_years, num_raw_materials, num_elements))

# X81 - Kg
X81_TRE = np.zeros((num_years, num_raw_materials, num_elements))

# S (Stock) - Cars, Kg
S_TCG = np.zeros((num_years, num_cohorts, num_component_groups))                                                # M5_TCG
S_TC = np.zeros((num_years, num_cohorts)) # Cars                                                                # M5_TC
S_T = np.zeros((num_years, 1)) # Cars                                                                              # M5_T
S_TCA = np.zeros((num_years, num_cohorts, num_alloys))                                                          # M5_TCA
S_TE = np.zeros((num_years, num_elements))                                                                      # M5_TE
S_TCE = np.zeros((num_years, num_cohorts, num_elements))                                                        #M5_TCE

# APR (Alloy Production Recipe) - Kg
APR_adj_TRA = np.zeros((num_years, num_raw_materials, num_alloys))                                              # Yadj_TRA
APR_adj_RAE = np.zeros((num_raw_materials, num_alloys, num_elements))                                           # Yadj_RAE
APR_RA = np.zeros((num_raw_materials, num_alloys))                                                              # Y_RA
APR_RAE = np.zeros((num_raw_materials, num_alloys, num_elements))                                               # Y_RAE
APR_TRA = np.zeros((num_years, num_raw_materials, num_alloys))                                                  # Y_TRA

# SRM (Suppy of Raw Materials) - Kg
SRM_TRE = np.zeros((num_years, num_raw_materials, num_elements))                                                # U_TRE
SRM_TR = np.zeros((num_years, num_raw_materials))                                                               # U_TR
SRM_conc_TRE = np.zeros((num_years, num_raw_materials, num_elements))                                           # U_TRe
for y in range(num_years):
    SRM_conc_TRE[y][:num_elements + 1] = primary_metal_comp
    SRM_TRE[y][:num_elements + 1] = [[math.inf] * len(primary_metal_comp.columns)] * len(primary_metal_comp.index)
    SRM_TR[y] = np.sum(SRM_TRE[y], axis = 1)
    if (demag_scen == 2):
        SRM_conc_TRE[y][num_elements] = [0] * num_elements
        SRM_TR[y][num_elements] = 0

# D (amount of magnesium removed)
D_TRA = np.zeros((num_years, num_raw_materials, num_alloys))                                                    # demagg_TRA
D_TA = np.zeros((num_years, num_alloys))                                                                        # demagg_TA
D_T = np.zeros(num_years)                                                                                       # demagg_T
D_TAE = np.zeros((num_years, num_alloys, num_elements))                                                         # demagg_TAE

# shred_contam (shredder contamination)
shred_contam = np.zeros(num_years)                                                                              # shredder_contamination                                                                                      
                                                                                                            
##### Composition of Alloys #####
# AC (alloy compositon)
AC_upper_AE = np.zeros((num_alloys, num_elements))                                                              # CAU_Ae
AC_lower_AE = np.zeros((num_alloys, num_elements))                                                              # CAL_Ae
for i_a, a in enumerate(alloy_names):
    a_df = alloy_data[alloy_data['Alloy'] == a] # equivalent to open_alloy(alloy_name, structure)
    for i_e, e in enumerate(element_names[1:]):
        if float(a_df[a_df['Level'] == 'Nominal'][e]) == 0:
            AC_upper_AE[i_a][i_e] = float(a_df[a_df['Level'] == 'Maximum'][e]) / 100
            AC_lower_AE[i_a][i_e] = 0
        else:
            AC_upper_AE[i_a][i_e] = float(a_df[a_df['Level'] == 'Nominal'][e]) / 100
            AC_lower_AE[i_a][i_e] = float(a_df[a_df['Level'] == 'Nominal'][e]) / 100
    AC_upper_AE[i_a][0] = 1 - sum(AC_lower_AE[i_a][1:])
    AC_lower_AE[i_a][0] = 1 - sum(AC_upper_AE[i_a][1:])
# set Al minimum content in 1070A to be .997 for some reason?

## Skeleton Stock-Driven Model

In [155]:
def normalDistTrunc(mu, sigma, time): # normal_distribution_trunc0
    dist = np.zeros((time, 1))
    for y in range(time):
        dist[y] = norm.cdf(y + 1, mu, sigma) - norm.cdf(y, mu, sigma)
    dist /= (1 - norm.cdf(0, mu, sigma))
    return dist

def stockDrivenModel(stock, dist): # stock_driven_model
    time = len(stock)
    stock_change = np.zeros((time, 1))
    stock_change[0] = stock[0]
    inp = np.zeros((time,1))
    inp[0] = stock[0]
    out = np.zeros((time, 1))
    out_cohort = np.zeros((time, time))
    stock_cohort = np.zeros((time, time))
    stock_cohort[0][0] = inp[0]
    
    for ot in range(1, time): # output time
        stock_change[ot] = stock[ot] - stock[ot - 1]
        for it in range(ot): # input time
            out_cohort[ot][it] = inp[it] * dist[ot - it - 1]
        out[ot] = np.sum(out_cohort[ot])
        inp[ot] = stock_change[ot] + out[ot]
        for ct in range(ot + 1): # cohort time
            #print(ot, sum(out_cohort[:(ot + 1)][ct]))
            stock_cohort[ot][ct] = inp[ct] - sum(out_cohort[:ot + 1, ct])
    return inp, out, out_cohort, stock_cohort

In [220]:
# Stock Calculation
S_T = np.multiply(cars_per_capita, population)

prob_dist = normalDistTrunc(exp_life, std_dev, num_years)
input_T, output_T, X560_TC, S_TC = stockDrivenModel(S_T, prob_dist)

# X45 - Segmented Input
X45_TS = input_T # 151 by 1, rather than 151 by 5 with columns 1-4 being zeros

for t in range(num_years):
    # input of groups - Kg (where GW refers to group weight)
    X34_TG[t] = np.multiply(np.squeeze(avg_Al_weight_per_comp.loc[t]).transpose() * np.squeeze(X45_TS[t].transpose()), 1.8)
    X45_TG[t] = np.divide(X34_TG[t], 1.8) # why 1.8
    X40_TG[t] = X34_TG[t] - X45_TG[t]
    # flow of material to manufacturing of component groups - Kg
    X23_TG[t] = np.divide(X34_TG[t], manufacture_yields.transpose())
    X301_TG[t] = X23_TG[t] - X34_TG[t]
    # flow of material to alloys of component groups - Kg
    X23_TGA[t] = np.diag(X23_TG[t]).dot(group_comp_by_alloy)
    X31_TAR[t] = (manufacture_scrap_recipe.transpose().dot(np.squeeze(X301_TGA[t]))).transpose()

X45_T = np.squeeze(np.sum(X45_TG, 1))
X40_T = np.squeeze(np.sum(X40_TG, 1))


for t in range(num_years):
    for c in range(t + 1):
        X560_TCG[t][c] = np.squeeze(X560_TC[t][c] * (segmentation[c] * np.squeeze(avg_Al_weight_per_comp.iloc[c])))
        X560_TCA[t][c] = np.dot(group_comp_by_alloy.transpose(), np.squeeze(X560_TCG[t][c])).transpose()
        X50_TCG[t][c] = X560_TCG[t][c] * (1 - collection_rate[t])
        S_TCG[t][c] = np.dot(np.dot(np.squeeze(S_TC[t][c]), segmentation[c]), np.squeeze(avg_Al_weight_per_comp.iloc[c])) # incorrect
        S_TCA[t][c] = np.dot(group_comp_by_alloy.transpose(), np.squeeze(S_TCG[t][c])).transpose() # incorrect because of S_TC
        

# X560_TCG (available scrap at EOL), X50_TCG (lost scrap), S_TCG (output and stocks as groups) - Kg
X56_TCG = X560_TCG - X50_TCG
X560_T = np.squeeze(np.sum(np.sum(X56_TCG, 2), 1))

for t in range(num_years):
    # X67a_TCG - dismantled parts
    X67a_TCG[t] = np.dot(np.squeeze(X56_TCG[t]), np.diag(np.squeeze(ELV_by_comp.iloc[t]))) # sum difference: 2.2618 vs 2.2499 (e+10)
    for c in range(t):
        # X67b_TCA - shredded alloys
        X67b_TCA[t][c] = np.dot(np.squeeze(X56_TCG[t][c] - X67a_TCG[t][c]).transpose(), group_comp_by_alloy) # sum difference: 1.4633 vs 1.4645 (e+11)
        # X67a_TCGA - dismantled alloys per cohort and group
        X67a_TCGA[t][c] = np.dot(np.diag(np.squeeze(X67a_TCG[t][c])), group_comp_by_alloy) # sum difference: 2.2618 vs 2.2499 (e+10)

22618453692.485382